In [1]:
import pandas as pd 
import numpy as np
rating=pd.read_csv('rating.csv')
rating['user_id']=rating['user_id'].astype(str)
print('mean ',rating.rating.mean())
print('median ',rating.rating.quantile(0.5))
print('lower boundary ',rating.rating.min())
print('upper boundary',rating.rating.max())
rating.head(5)

mean  6.144029546937656
median  7.0
lower boundary  -1
upper boundary 10


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


<h3>Cut animes that have audiences under 100

In [21]:
anime_popular=rating.groupby(['anime_id'])['user_id'].nunique().to_frame().sort_values('user_id',ascending=False)
anime_popular.rename(columns={'user_id':'#users'},inplace=True)
anime_popular

,#users
anime_id,
1535,39340
11757,30582
16498,29583
1575,27717
226,27506
...,...
23865,1
23863,1
23861,1


In [22]:
anime_list=anime_popular[anime_popular['#users']>100].index
anime_list

Int64Index([ 1535, 11757, 16498,  1575,   226,  6547,    20,   121,  5114,
             4224,
            ...
            28779,  2943,  2713,   827,  8289, 11745,  2591,  1777,  3886,
             8234],
           dtype='int64', name='anime_id', length=4575)

In [24]:
rating_cut=rating[rating['anime_id'].isin(anime_list)]
print(rating_cut.shape)
rating_cut.head(3)

(7670697, 3)


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1


In [ ]:
anime=pd.read_csv('anime.csv')
print(anime.shape)

In [25]:
anime_rating=rating_cut.groupby(['anime_id'])['user_id'].nunique().to_frame().sort_values('user_id',ascending=False)
anime_rating

,user_id
anime_id,
1535,39340
11757,30582
16498,29583
1575,27717
226,27506
...,...
2591,101
20907,101
827,101


In [26]:
rating_pivot=pd.pivot_table(rating,index='user_id',columns='anime_id',values='rating')
rating_pivot

anime_id,1,5,6,7,8,15,16,17,18,19,...,34283,34324,34325,34349,34358,34367,34412,34475,34476,34519
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,10.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
def highlight(val):
    yellow = 'background-color: yellow'
    return yellow if val>0 else ''

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

user=input('user')
rating_pivot.loc[[user]].style.applymap(highlight)

user1


anime_id,1,5,6,7,8,15,16,17,18,19,20,22,23,24,25,26,27,28,29,30,31,32,33,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,71,72,73,74,75,76,77,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,225,226,227,228,229,230,231,232,233,234,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,411,412,413,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,481,482,483,484,485,486,487,488,489,490,492,493,495,496,497,498,499,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,543,544,545,546,547,548,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,576,577,578,579,580,581,582,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,629,630,631,632,633,634,635,636,637,641,642,644,645,646,647,648,649,650,651,652,653,654,655,656,658,659,660,661,664,665,666,667,668,669,670,671,673,675,676,677,678,679,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,704,705,706,707,708,709,710,711,712,713,714,715,718,719,721,722,723,724,725,727,729,730,731,732,733,734,736,738,740,741,743,744,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,769,770,771,773,776,777,779,780,781,782,783,785,786,788,789,790,791,792,793,795,796,797,798,799,800,801,807,808,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,827,828,830,831,832,833,834,835,836,837,838,839,840,841,842,844,845,846,848,849,850,851,852,853,854,855,856,857,858,859,860,861,863,864,865,866,867,868,869,870,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,924,925,926,927,928,929,930,932,933,934,935,936,937,938,940,941,942,943,944,949,950,951,952,953,956,957,958,959,960,961,962,963,964,965,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1055,1056,1057,1058,1059,1060,1062,1064,1065,1066,1067,1068,1069,1070,1071,1072,1074,1075,1076,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1

In [29]:
target=rating_pivot.loc[[user]].T
target=target[target[user].notnull()].T.columns
target

Int64Index([   20,    24,    79,   226,   241,   355,   356,   442,   487,
              846,
            ...
            27899, 28121, 28677, 29093, 29095, 30015, 30296, 30544, 31338,
            31845],
           dtype='int64', name='anime_id', length=153)

In [30]:
rating_compare=rating_pivot[target]
rating_compare.fillna(0,inplace=True)
cosine=cosine_similarity(rating_compare,rating_pivot[target].loc[[user]])
cosine_user=pd.DataFrame(cosine,index=rating_compare.index).sort_values(0,ascending=False)
cosine_user.rename(columns={0:user},inplace=True)
cosine_user

C:\Users\aomsi\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,1
user_id,
1,1.000000
64220,0.841652
64491,0.772854
9825,0.744378
69976,0.739477
...,...
48413,-0.534425
69067,-0.596852
63727,-0.603572


In [31]:
sim_users=cosine_user[cosine_user[user]>0.6].index
sim_user_df=rating_pivot.loc[sim_users]
sim_user_df=sim_user_df.notnull().astype(int)
sim_user_df

anime_id,1,5,6,7,8,15,16,17,18,19,...,34283,34324,34325,34349,34358,34367,34412,34475,34476,34519
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64491,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9825,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69976,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45520,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38068,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73023,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
target_user=rating_pivot.loc[user].notnull().astype(int)
rec_df=sim_user_df.subtract(target_user,axis=1)
rec_df=rec_df.T
rec_df.replace(-1,0,inplace=True)
rec_df

user_id,1,64220,64491,9825,69976,71834,9450,1776,46337,65558,...,72235,34207,15857,13380,14750,45520,38068,73023,46626,46667
anime_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34412,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34475,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
rec_df['weight_rec']=rec_df.sum(axis=1)
rec_df=rec_df[rec_df['weight_rec']>len(sim_users)/10].sort_values('weight_rec',ascending=False)
rec_df[['weight_rec']].style.background_gradient(subset=['weight_rec'])

user_id,weight_rec
anime_id,
1535,140
24703,84
121,77
10620,63
5114,56
813,56
11111,56
3588,56
6880,42


In [32]:
def RecommendCosineItem(user):
    user=str(user)
    target=rating_pivot.loc[[user]].T
    target=target[target[user].notnull()].T.columns
    
    rating_compare=rating_pivot[target]
    rating_compare.fillna(0,inplace=True)
    cosine=cosine_similarity(rating_compare,rating_pivot[target].loc[[user]])
    cosine_user=pd.DataFrame(cosine,index=rating_compare.index).sort_values(0,ascending=False)
    cosine_user.rename(columns={0:user},inplace=True)
    
    sim_users=cosine_user[cosine_user[user]>0.6].index
    sim_user_df=rating_pivot.loc[sim_users]
    sim_user_df=sim_user_df.notnull().astype(int)

    target_user=rating_pivot.loc[user].notnull().astype(int)
    rec_df=sim_user_df.subtract(target_user,axis=1)
    rec_df=rec_df.T
    rec_df.replace(-1,0,inplace=True)
    return cosine_user

In [33]:
RecommendCosineItem('1')

C:\Users\aomsi\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,1
user_id,
1,1.000000
64220,0.841652
64491,0.772854
9825,0.744378
69976,0.739477
...,...
48413,-0.534425
69067,-0.596852
63727,-0.603572
